In [13]:
import pandas as pd

In [14]:
df=pd.read_csv('train/train.csv')

In [15]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
df=df.dropna()

In [17]:
X=df.drop('label',axis=1)


In [18]:
y=df['label']

In [19]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
voc_size=5000
messages=X.copy()
messages.reset_index(inplace=True)

In [9]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
from tensorflow.keras.preprocessing.text import one_hot

In [22]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 20
embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_doc)

[[2781 1389 2701 ... 1655 4882 3310]
 [2636  446 1172 ... 1065 2360 4466]
 [3164 1918 2940 ... 4020 2629  641]
 ...
 [4427 3766 3069 ... 3074 3557 3362]
 [2435 2372 4411 ... 3993  913 2922]
 [2232 1806 1518 ... 4241 4320  155]]


In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [25]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.2))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.2))
model1.add(Dense(2,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 402       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 313,205
Trainable params: 313,205
Non-trainable params: 0
__________________________________________________

In [26]:
import numpy as np
X_final=np.array(embedded_doc)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
192/192 [==============================] - 33s 89ms/step - loss: 0.4280 - accuracy: 0.7874 - val_loss: 0.3339 - val_accuracy: 0.8482
Epoch 2/15
192/192 [==============================] - 13s 66ms/step - loss: 0.2580 - accuracy: 0.8912 - val_loss: 0.3605 - val_accuracy: 0.8462
Epoch 3/15
192/192 [==============================] - 9s 48ms/step - loss: 0.1981 - accuracy: 0.9194 - val_loss: 0.3522 - val_accuracy: 0.8495
Epoch 4/15
192/192 [==============================] - 9s 48ms/step - loss: 0.1500 - accuracy: 0.9433 - val_loss: 0.4085 - val_accuracy: 0.8381
Epoch 5/15
192/192 [==============================] - 10s 51ms/step - loss: 0.1143 - accuracy: 0.9577 - val_loss: 0.4851 - val_accuracy: 0.8389
Epoch 6/15
192/192 [==============================] - 9s 48ms/step - loss: 0.0991 - accuracy: 0.9647 - val_loss: 0.5483 - val_accuracy: 0.8374
Epoch 7/15
192/192 [==============================] - 9s 48ms/step - loss: 0.0791 - accuracy: 0.9727 - val_loss: 0.5457 - val_accuracy: 0.8

In [30]:
y_pred1=model1.predict_classes(X_test)

C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test,y_pred1)

array([[2965,  454],
       [ 557, 2059]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.8324772162386082

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      3419
           1       0.81      0.79      0.80      2616

    accuracy                           0.83      6035
   macro avg       0.83      0.82      0.83      6035
weighted avg       0.83      0.83      0.83      6035

